# Setup

In [1]:
! uv pip install agentics-py

Using Python 3.12.12 environment at: /usr
Audited 1 package in 535ms


In [2]:
import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

In Colab: True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Setup Custom LLM

In [10]:
from crewai import LLM

# pick a provider (openai, anthropic, groq, etc.) - see crewai docs for details
gemini_llm = LLM(
    model="gemini/gemini-2.0-flash-lite",
    temperature=0.7,  # Adjust based on task
    max_tokens=4096,  # Set based on output needs
    # timeout=300,
)  # Longer timeout for complex tasks

print(gemini_llm)

In [11]:
# test call
print(gemini_llm.call("where is the Eiffel Tower?"))

The Eiffel Tower is located in **Paris, France**.



# Download HF Dataset

In [12]:
from datasets import load_dataset, DatasetDict
import warnings
warnings.filterwarnings("ignore")

DATASET_NAME = "NickyNicky/finance-financialmodelingprep-stock-news-sentiments-rss-feed"
SPLIT_NAME = "train"

In [13]:
def download_dataset(dataset_name: str, split_name: str) -> DatasetDict:
  try:
    # The load_dataset function downloads the dataset to your local cache
    # and loads it into memory as a Dataset object.
    dataset = load_dataset(dataset_name, split=split_name)

    print("\n--- Download Successful! ---")
    print(f"Loaded dataset type: {type(dataset)}")

    # Print basic information
    print(f"\nTotal number of rows in the '{split_name}' split: {len(dataset)}")
    print("\nFeatures (columns) in the dataset:")
    print(dataset.column_names)
    return dataset
  except FileNotFoundError:
      print(f"Error: Dataset or split '{dataset_name}/{split_name}' not found on the Hub.")
      print("Please check the dataset name and split name for typos.")
  except Exception as e:
      print(f"An unexpected error occurred during dataset loading: {e}")

ds = download_dataset(DATASET_NAME, SPLIT_NAME)


--- Download Successful! ---
Loaded dataset type: <class 'datasets.arrow_dataset.Dataset'>

Total number of rows in the 'train' split: 142000

Features (columns) in the dataset:
['symbol', 'publishedDate', 'title', 'image', 'site', 'text', 'url', 'sentiment', 'sentimentScore']


In [14]:
ds[101010]

{'symbol': 'BTC-USD',
 'publishedDate': '2022-11-29T18:40:15.000Z',
 'title': '$16K Bitcoin dropping to $12K–$14K — Can this really happen? Watch The Market Report',
 'image': 'https://i-invdn-com.investing.com/news/LYNXNPEE1P15Z_L.jpg',
 'site': 'investing',
 'text': '$16K Bitcoin dropping to $12K–$14K — Can this really happen? Watch The Market Report     \xa0      BTC/USD +1.77%      Add to/Remove from Watchlist          Add to Watchlist   Add Position     Position added successfully to:            Please name your holdings portfolio     Type:  BUY SELL    Date:           \xa0    Amount:    Price    Point Value:       Leverage:  1:1 1:10 1:25 1:50 1:100 1:200 1:400 1:500 1:1000    Commission:          \xa0  Create New Watchlist Create    Create a new holdings portfol',
 'url': 'https://www.investing.com/news/cryptocurrency-news/16k-bitcoin-dropping-to-12k14k--can-this-really-happen-watch-the-market-report-2953888',
 'sentiment': 'Positive',
 'sentimentScore': 0.9231}

# Setup Custom Tools
Todo: Used to get news for inference, not for training.

In [15]:
# from crewai.tools import tool
# from ddgs import DDGS


# ## Define a Crew AI tool to get news for a given date using the DDGS search engine
# @tool("web_search")
# def web_search(query: str) -> str:
#     """Fetch web search results for the given query using DDGS."""
#     return str(DDGS().text(query, max_results=10))


# questions_answering_ag.verbose_agent = True
# questions_answering_ag.tools = [web_search]
# dow_jones_data.filter_states(end=1)
# answers = await (questions_answering_ag << dow_jones_data)
# print(answers.pretty_print())

# Transduction between AGs
source and target AGs   
News -> Classify News + Key points

**Customizing Transduction**   
You can fine-tune how logical transduction works by configuring:

LLMs – choose the underlying language model to run the transduction.   
Instructions – add task-specific guidance for the LLM.   
Prompt Templates – control how inputs are rendered into prompts.   
Few-Shot Examples – provide examples to steer the model’s behavior.   
Verbose Options – enable detailed logging and debug outputs.



In [16]:
from typing import Optional, Literal, List, Dict, Any
from pydantic import BaseModel, Field, HttpUrl, ValidationError
from datasets import load_dataset, DatasetDict
from pydantic import BaseModel
from agentics import AG
import asyncio

In [38]:
INDUSTRIES = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Consumer Discretionary",
    "Communication Services",
    "Industrials",
    "Consumer Staples",
    "Energy",
    "Utilities",
    "Real Estate",
    "Materials",
    "General Market"
]

IndustryType = Literal[tuple(INDUSTRIES)]

class NewsEntry(BaseModel):
    """
    Pydantic model representing a single comprehensive news entry, including
    sentiment analysis and source metadata.
    """
    # url: HttpUrl = Field(description="The full URL link to the original news article.")
    # image: Optional[FlexibleImageUrl] = Field(description="Optional URL link to the main image or a relative path.")
    publishedDate: str = Field(description="The publication date of the article (ISO 8601 string).")
    symbol: str = Field(description="The stock ticker symbol the news is related to (e.g., 'AAPL', 'OCEA').")
    site: str = Field(description="The source website or platform where the news was published.")
    text: str = Field(description="A short snippet or summary of the news article content.")
    title: str = Field(description="The headline or title of the news article.")
    sentiment: Optional[str] = Field(description="The calculated sentiment label, usually one of 'positive', 'negative', or 'neutral'.")
    sentimentScore: Optional[float] = Field(description="The numeric sentiment score calculated by the source.")
    # New additions
    industry: IndustryType = Field(None, description=f"The primary industry sector this news is relevant to. Must be one of: {INDUSTRIES}.")
    keyPoints: str = Field(None, description="A bullet list summarizing the 5 most important points of the news article.")

In [39]:
news_entry_instance = NewsEntry.model_validate(ds[101010])
print(news_entry_instance)

publishedDate='2022-11-29T18:40:15.000Z' symbol='BTC-USD' site='investing' text='$16K Bitcoin dropping to $12K–$14K — Can this really happen? Watch The Market Report     \xa0      BTC/USD +1.77%      Add to/Remove from Watchlist          Add to Watchlist   Add Position     Position added successfully to:            Please name your holdings portfolio     Type:  BUY SELL    Date:           \xa0    Amount:    Price    Point Value:       Leverage:  1:1 1:10 1:25 1:50 1:100 1:200 1:400 1:500 1:1000    Commission:          \xa0  Create New Watchlist Create    Create a new holdings portfol' title='$16K Bitcoin dropping to $12K–$14K — Can this really happen? Watch The Market Report' sentiment='Positive' sentimentScore=0.9231 industry=None keyPoints=None


In [40]:
from typing import Optional, Literal, List, Dict, Any, Tuple
from pydantic import BaseModel, Field, HttpUrl, ValidationError
from datasets import load_dataset, DatasetDict
from pydantic import BaseModel
from agentics import AG
import asyncio

async def process_hf_dataset(hf_dataset: List[Dict[str, Any]]) -> List[NewsEntry]:
  """
  Processes a list of data (simulating a Hugging Face dataset split)
  to validate each entry against the NewsEntry Pydantic model.
  """
  print(f"\n--- Starting processing of {len(hf_dataset)} entries ---")

  # Validate each record synchronously
  results = [NewsEntry.model_validate(record) for record in hf_dataset]
  print("--- Processing Complete! ---")
  return results

In [41]:
news_entry = await process_hf_dataset(ds.select(range(5))) # to remove, testing on 5 now


--- Starting processing of 5 entries ---
--- Processing Complete! ---


In [42]:
# news_entry

In [43]:
prompts = {
    "industry_class_and_keypoints": f"""Classify the primary industry sector this news is relevant to as \"industry\". Must be one of: {INDUSTRIES}.
    Next, summarize the 5 most important points of the news article in bullet point as \"keyPoints\"."""
}

In [44]:
# Create source and target AGs
ag_news_entry = AG(atype=NewsEntry, states=news_entry)
# ag_news_kp = AG(atype=NewsKeyPoints, llm=gemini_llm)

# ag_news_entry.instructions = prompts["industry_class_and_keypoints"]
ag_news_entry.llm = (gemini_llm)
ag_news_entry = await ag_news_entry.self_transduction(
    source_fields=list(NewsEntry.model_fields.keys()),
    target_fields=["industry", "keyPoints"],
    instructions= prompts["industry_class_and_keypoints"]
)

Output()

In [47]:
ag_news_entry.pretty_print()

aType : <class '__main__.NewsEntry'>
publishedDate: '2023-10-04T21:54:28.000Z'
symbol: HE
site: benzinga
text: RADNOR, Pa., Oct. 04, 2023 (GLOBE NEWSWIRE) -- The law firm of Kessler Topaz
  Meltzer & Check, LLP (www.ktmc.com) informs investors that a securities class action
  lawsuit has been filed in the United States District Court for the Northern District
  of California against Hawaiian Electric Industries, Inc. ("Hawaiian Electric") (NYSE:HE).
  The action charges Hawaiian Electric with violations of the federal securities laws,
  including omissions and fraudulent misrepresentations relating to the compa...
title: 'HE REMINDER: Kessler Topaz Meltzer & Check, LLP Reminds Hawaiian Electric
  Industries, Inc. (HE) Shareholders of Securities Fraud Class Action Lawsuit and
  Encourages Investors with Substantial Losses to Contact the Firm'
sentiment: Negative
sentimentScore: -0.6417
industry: Utilities
keyPoints: '- A securities class action lawsuit has been filed against Hawaiian El

'aType : <class \'__main__.NewsEntry\'>\npublishedDate: \'2023-10-04T21:54:28.000Z\'\nsymbol: HE\nsite: benzinga\ntext: RADNOR, Pa., Oct. 04, 2023 (GLOBE NEWSWIRE) -- The law firm of Kessler Topaz\n  Meltzer & Check, LLP (www.ktmc.com) informs investors that a securities class action\n  lawsuit has been filed in the United States District Court for the Northern District\n  of California against Hawaiian Electric Industries, Inc. ("Hawaiian Electric") (NYSE:HE).\n  The action charges Hawaiian Electric with violations of the federal securities laws,\n  including omissions and fraudulent misrepresentations relating to the compa...\ntitle: \'HE REMINDER: Kessler Topaz Meltzer & Check, LLP Reminds Hawaiian Electric\n  Industries, Inc. (HE) Shareholders of Securities Fraud Class Action Lawsuit and\n  Encourages Investors with Substantial Losses to Contact the Firm\'\nsentiment: Negative\nsentimentScore: -0.6417\nindustry: Utilities\nkeyPoints: \'- A securities class action lawsuit has been f